In [25]:
CapacityMetricsWorkspace = 'Microsoft Fabric Capacity Metrics'
CapacityMetricsDataset = 'Fabric Capacity Metrics'

StatementMeta(, bc92e970-394c-4344-bf51-a281977ce0fd, 27, Finished, Available, Finished)

### Get the workspace id

In [26]:
workspaceId = spark.conf.get('trident.workspace.id')
print(f"{workspaceId=}")

StatementMeta(, bc92e970-394c-4344-bf51-a281977ce0fd, 28, Finished, Available, Finished)

workspaceId='65ba0c17-9fc7-4566-a01a-78da40dbe5f4'


In [27]:
import requests, json

header = {'Authorization': f'Bearer {mssparkutils.credentials.getToken("pbi")}'
          ,"Content-Type": "application/json"
          }

response = requests.request(method='get', url=f'https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}', headers=header)
workspaceName = response.json().get('displayName')
print(f'{workspaceName=}')

StatementMeta(, bc92e970-394c-4344-bf51-a281977ce0fd, 29, Finished, Available, Finished)

workspaceName='WS_TESTING_DEPLOYMENT'


### Create lakehouses

In [28]:
from notebookutils import mssparkutils

for lakehouseName in ['LH_SampleData', 'LH_QueryResults']:
    try:
        lakehouseItem = mssparkutils.lakehouse.create(lakehouseName, "", workspaceId)
    except Exception as e:
        # print(e)
        pass

LH_SampleData = mssparkutils.lakehouse.get("LH_SampleData", workspaceId)
LH_QueryResults = mssparkutils.lakehouse.get("LH_QueryResults", workspaceId)
print(f'{LH_SampleData=}')
print(f'{LH_QueryResults=}')

StatementMeta(, bc92e970-394c-4344-bf51-a281977ce0fd, 30, Finished, Available, Finished)

LH_SampleData={'id': 'b801f394-c183-4b96-a0b5-f694236f3b7f', 'type': 'Lakehouse', 'displayName': 'LH_SampleData', 'description': '', 'workspaceId': '65ba0c17-9fc7-4566-a01a-78da40dbe5f4', 'properties': {'abfsPath': 'abfss://65ba0c17-9fc7-4566-a01a-78da40dbe5f4@onelake.dfs.fabric.microsoft.com/b801f394-c183-4b96-a0b5-f694236f3b7f'}}
LH_QueryResults={'id': 'cec6e34e-5f2e-4277-9fe7-4f9961006afc', 'type': 'Lakehouse', 'displayName': 'LH_QueryResults', 'description': '', 'workspaceId': '65ba0c17-9fc7-4566-a01a-78da40dbe5f4', 'properties': {'abfsPath': 'abfss://65ba0c17-9fc7-4566-a01a-78da40dbe5f4@onelake.dfs.fabric.microsoft.com/cec6e34e-5f2e-4277-9fe7-4f9961006afc'}}


### Get notebook definition from github

In [29]:
import requests, json, base64
url = "https://raw.githubusercontent.com/bretamyers/FabricTools/dev/FabricDWQueryCostAnalyzer/NB_DW_Load_Cost_Analyzer.ipynb"
response = requests.get(url)

b = base64.b64encode(bytes(json.dumps(response.json()), 'utf-8')) # convert to bytes
base64_str = b.decode('utf-8') # convert bytes to string
notebookDefDict = json.loads(base64.b64decode(base64_str).decode('utf-8')) # convert base64 bytes to string and then to dictionary
notebookDefDict['metadata']['dependencies']['lakehouse']['default_lakehouse'] = LH_QueryResults.get('id')
notebookDefDict['metadata']['dependencies']['lakehouse']['default_lakehouse_name'] = LH_QueryResults.get('displayName')
notebookDefDict['metadata']['dependencies']['lakehouse']['default_lakehouse_workspace_id'] = LH_QueryResults.get('workspaceId')
notebookDefBase64 = base64.b64encode(json.dumps(notebookDefDict).encode('utf-8')).decode('utf-8') # convert dictionary back to base64 string

StatementMeta(, bc92e970-394c-4344-bf51-a281977ce0fd, 31, Finished, Available, Finished)

### Create notebook from the definition

In [30]:
import requests, json, time, base64

header = {'Authorization': f'Bearer {mssparkutils.credentials.getToken("pbi")}'
          ,"Content-Type": "application/json"
          }

body = {
    "displayName": "NB_DW_Load_Cost_Analyzer",
    "description": "A notebook to run data warehouse queries and capture the duration and cost of those queries.",
    "definition": {
        "format": "ipynb",
        "parts": [
            {
                "path": "notebook-content.py"
                ,"payload": notebookDefBase64
                ,"payloadType": "InlineBase64"
            }
        ]
    }
}

response = requests.request(method='post', url=f'https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/notebooks', headers=header, data=json.dumps(body))

if response.status_code == 202:
    time.sleep(int(response.headers.get('Retry-After')))
    for retry in range(5):
        responseStatus = requests.request(method='get', url=response.headers.get('Location'), headers=header)
        print(responseStatus.text)
        print(responseStatus.headers)
        if responseStatus.headers.get('Location') is not None:
            break
        else:
            time.sleep(int(responseStatus.headers.get('Retry-After')))
else:
    print(f"Error - {response.json()}")

StatementMeta(, bc92e970-394c-4344-bf51-a281977ce0fd, 32, Finished, Available, Finished)

Error - {'requestId': '7367e1c3-6cf5-4c65-8b56-58b2b080160c', 'errorCode': 'ItemDisplayNameAlreadyInUse', 'message': "Requested 'NB_DW_Load_Cost_Analyzer' is already in use"}


### Get the notebook id

In [31]:
import requests, json

header = {'Authorization': f'Bearer {mssparkutils.credentials.getToken("pbi")}'
          ,"Content-Type": "application/json"
          }
          
# Need to implement for pagination
response = requests.request(method='get', url=f'https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/notebooks', headers=header)

for notebook in response.json().get('value'):
    if notebook.get('displayName') == 'NB_DW_Load_Cost_Analyzer':
        notebookId = notebook.get('id')
        print(f'{notebookId=}')


StatementMeta(, bc92e970-394c-4344-bf51-a281977ce0fd, 33, Finished, Available, Finished)

notebookId='b3931333-0cb7-4406-9e05-b3ea0ca0f681'


### Create Warehouse

In [32]:
import requests, json, time

body = {"displayName": "WH_SampleData"}

response = requests.request(method='post', url=f'https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/warehouses', headers=header, data=json.dumps(body))

if response.status_code == 202:
    print(f'Creating Warehouse {body.get("displayName")}')
    time.sleep(int(response.headers.get('Retry-After')))
    for retry in range(5):
        responseStatus = requests.request(method='get', url=response.headers.get('Location'), headers=header)
        print(responseStatus.text)
        print(responseStatus.headers)
        if responseStatus.json().get('status') != 'Succeeded':
            time.sleep(int(responseStatus.headers.get('Retry-After')))
        else:
            if responseStatus.headers.get('Location') is None: # no result was return but operation completed
                break
            else:
                responseResult = requests.request(method='get', url=responseStatus.headers.get('Location'), headers=header)
                print(f"Succeeded {responseResult.json()}")
                break
else:
    print(response.text)

StatementMeta(, bc92e970-394c-4344-bf51-a281977ce0fd, 34, Finished, Available, Finished)

{"requestId":"e2bd56fa-6bb5-42e8-ac85-40e573fdcebf","errorCode":"UnknownError","message":"An unexpected error occurred while processing the request"}


In [33]:
import requests, json, time

response = requests.request(method='get', url=f'https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/warehouses', headers=header)

for warehouse in response.json().get('value'):
    if warehouse.get('displayName') == 'WH_SampleData':
        WH_SampleData = warehouse
        break
print(WH_SampleData)

StatementMeta(, bc92e970-394c-4344-bf51-a281977ce0fd, 35, Finished, Available, Finished)

{'id': '6d26fa1e-f60f-4a4f-a9a1-89d387a64d87', 'type': 'Warehouse', 'displayName': 'WH_SampleData', 'description': '', 'workspaceId': '65ba0c17-9fc7-4566-a01a-78da40dbe5f4', 'properties': {'connectionInfo': 'n52dzjnhphme5jslxpytuo62ni-c4gluzoht5telia2pdnebw7f6q.datawarehouse.fabric.microsoft.com', 'connectionString': 'n52dzjnhphme5jslxpytuo62ni-c4gluzoht5telia2pdnebw7f6q.datawarehouse.fabric.microsoft.com', 'createdDate': '2024-09-25T12:49:21.977', 'lastUpdatedTime': '2024-09-25T12:49:35.407'}}


### Load data to Lakehouse

In [34]:
import azure.storage.blob
import pandas as pd
from notebookutils import mssparkutils

spark.conf.set('spark.sql.parquet.int96RebaseModeInWrite', 'LEGACY')
spark.conf.set('spark.sql.parquet.datetimeRebaseModeInWrite', 'LEGACY')

account_url = "https://fabrictutorialdata.blob.core.windows.net"
blob_service_client = azure.storage.blob.BlobServiceClient(account_url)
container_client = blob_service_client.get_container_client('sampledata')

tableList = list()
for blob in container_client.walk_blobs(name_starts_with='WideWorldImportersDW/parquet/tables/', delimiter='/'):
    tableName = blob.name.split('/')[-1].split('.parquet')[0]
    tableList.append(tableName)

for table in tableList:
    print(f'Loading Table: {table}')
    if mssparkutils.fs.exists(f"{LH_SampleData.get('properties').get('abfsPath')}/Tables/{table}"):
        mssparkutils.fs.rm(f"{LH_SampleData.get('properties').get('abfsPath')}/Tables/{table}", recurse=True)

    for blob in container_client.walk_blobs(name_starts_with=f"WideWorldImportersDW/parquet/tables/", delimiter='/'):
        if f'{table}.parquet' == blob.name.split('/')[-1]:
            dfPandas = pd.read_parquet(f'{account_url}/sampledata/{blob.name}', engine='pyarrow', storage_options={'anon': True})

            if not dfPandas.empty:
                dfSpark = spark.createDataFrame(dfPandas)
                dfSpark.write.format('delta').mode('append').option('mergeSchema', "true").save(f"{LH_SampleData.get('properties').get('abfsPath')}/Tables/{table}")
            else:
                print('Bad file', blob.name)

StatementMeta(, bc92e970-394c-4344-bf51-a281977ce0fd, 36, Finished, Available, Finished)

Loading Table: DimCity
Loading Table: DimCustomer
Loading Table: DimDate
Loading Table: DimEmployee
Loading Table: DimPaymentMethod
Loading Table: DimStockItem
Loading Table: DimSupplier
Loading Table: DimTransactionType
Loading Table: FactMovement
Loading Table: FactOrder
Loading Table: FactPurchase
Loading Table: FactSale
Loading Table: FactStockHolding
Loading Table: FactTransaction


### Load data to Warehouse

In [35]:
from notebookutils import mssparkutils  
from pyspark.sql import functions as F
from pyspark.sql import Row
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType, TimestampType, DoubleType
import pyodbc, struct, itertools, time, datetime, re, uuid, json

connectionString = f'DRIVER={{ODBC Driver 18 for SQL Server}};SERVER={WH_SampleData.get("properties").get("connectionString")};Database={WH_SampleData.get("displayName")}'

# Use the credentials of the user executing the notebook
token = bytes(mssparkutils.credentials.getToken('pbi'), "UTF-8")
encoded_bytes = bytes(itertools.chain.from_iterable(zip(token, itertools.repeat(0))))
tokenstruct = struct.pack("<i", len(encoded_bytes)) + encoded_bytes

def get_result_set(cursor):
    if cursor.description:
        resultList = cursor.fetchall()
        resultColumns = columns = [column[0] for column in cursor.description]
    else:
        resultList = []
        resultColumns = []
    return [dict(zip(resultColumns, [str(col) for col in row])) for row in resultList]

with pyodbc.connect(connectionString, attrs_before = { 1256:tokenstruct }) as conn:
    with conn.cursor() as cursor:
        for retry in range(5):
            cursor.execute(f"""SELECT TABLE_NAME FROM LH_SampleData.INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'dbo'""")
            resultSet = get_result_set(cursor)
            if len(resultSet) == 14:
                break
            else:
                time.sleep(30) # wait 30 seconds for the schema sync to occur
        while cursor.nextset():
            queryMessage += ",".join([str(cursor.messages) if cursor.messages else ""])
            resultSet.append(get_result_set(cursor))
        cursor.commit()

    for table in resultSet:
        with conn.cursor() as cursor:
            query = f"""IF OBJECT_ID('dbo.{table.get("TABLE_NAME")}', 'U') IS NOT NULL DROP TABLE dbo.{table.get("TABLE_NAME")}; CREATE TABLE dbo.{table.get("TABLE_NAME")} AS SELECT * FROM LH_SampleData.dbo.{table.get("TABLE_NAME")}"""
            print(query)
            cursor.execute(query)
            cursor.commit()


StatementMeta(, bc92e970-394c-4344-bf51-a281977ce0fd, 37, Finished, Available, Finished)

IF OBJECT_ID('dbo.FactOrder', 'U') IS NOT NULL DROP TABLE dbo.FactOrder; CREATE TABLE dbo.FactOrder AS SELECT * FROM LH_SampleData.dbo.FactOrder
IF OBJECT_ID('dbo.DimDate', 'U') IS NOT NULL DROP TABLE dbo.DimDate; CREATE TABLE dbo.DimDate AS SELECT * FROM LH_SampleData.dbo.DimDate
IF OBJECT_ID('dbo.FactStockHolding', 'U') IS NOT NULL DROP TABLE dbo.FactStockHolding; CREATE TABLE dbo.FactStockHolding AS SELECT * FROM LH_SampleData.dbo.FactStockHolding
IF OBJECT_ID('dbo.FactSale', 'U') IS NOT NULL DROP TABLE dbo.FactSale; CREATE TABLE dbo.FactSale AS SELECT * FROM LH_SampleData.dbo.FactSale
IF OBJECT_ID('dbo.DimCustomer', 'U') IS NOT NULL DROP TABLE dbo.DimCustomer; CREATE TABLE dbo.DimCustomer AS SELECT * FROM LH_SampleData.dbo.DimCustomer
IF OBJECT_ID('dbo.DimSupplier', 'U') IS NOT NULL DROP TABLE dbo.DimSupplier; CREATE TABLE dbo.DimSupplier AS SELECT * FROM LH_SampleData.dbo.DimSupplier
IF OBJECT_ID('dbo.FactMovement', 'U') IS NOT NULL DROP TABLE dbo.FactMovement; CREATE TABLE dbo.Fa

In [36]:
from notebookutils import mssparkutils  
from pyspark.sql import functions as F
from pyspark.sql import Row
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType, TimestampType, DoubleType
import pyodbc, struct, itertools, time, datetime, re, uuid, json

connectionString = f'DRIVER={{ODBC Driver 18 for SQL Server}};SERVER={WH_SampleData.get("properties").get("connectionString")};Database={WH_SampleData.get("displayName")}'

# Use the credentials of the user executing the notebook
token = bytes(mssparkutils.credentials.getToken('pbi'), "UTF-8")
encoded_bytes = bytes(itertools.chain.from_iterable(zip(token, itertools.repeat(0))))
tokenstruct = struct.pack("<i", len(encoded_bytes)) + encoded_bytes

def get_result_set(cursor):
    if cursor.description:
        resultList = cursor.fetchall()
        resultColumns = columns = [column[0] for column in cursor.description]
    else:
        resultList = []
        resultColumns = []
    return [dict(zip(resultColumns, [str(col) for col in row])) for row in resultList]

with pyodbc.connect(connectionString, attrs_before = { 1256:tokenstruct }) as conn:
    with conn.cursor() as cursor:
        cursor.execute('''IF OBJECT_ID('dbo.sp_Query', 'P') IS NOT NULL
	DROP PROCEDURE dbo.sp_Query
;''')
        cursor.execute('''CREATE PROCEDURE dbo.sp_Query AS
	SELECT COUNT(*) AS cnt_FactOrder FROM FactOrder

    SELECT COUNT(*) AS cnt_FactMovement FROM FactMovement

    SELECT COUNT(*) AS cnt_FactPurchase FROM FactPurchase

;''')
        cursor.execute('''IF OBJECT_ID('dbo.sp_Ingest', 'P') IS NOT NULL
	DROP PROCEDURE dbo.sp_Ingest
;''')
        cursor.execute('''CREATE PROCEDURE dbo.sp_Ingest AS
        IF OBJECT_ID('dbo.DimCity', 'U') IS NOT NULL DROP TABLE dbo.DimCity; CREATE TABLE dbo.DimCity AS SELECT * FROM LH_SampleData.dbo.DimCity;

        SELECT COUNT(*) FROM DimCity
;''')
        cursor.commit()


StatementMeta(, bc92e970-394c-4344-bf51-a281977ce0fd, 38, Finished, Available, Finished)

### Refresh Fabric Capacity Metrics App so that the latest workspace artifacts are in the model

In [37]:
import requests, json, time

def model_refresh():
    header = {'Authorization': f'Bearer {mssparkutils.credentials.getToken("pbi")}'
            ,"Content-Type": "application/json"
            }

    response = requests.get('https://api.fabric.microsoft.com/v1/workspaces', headers=header)

    capacityWorkspaceId = [workspace.get('id') for workspace in response.json().get('value') if workspace.get('displayName') == CapacityMetricsWorkspace][0]
    print(f'{capacityWorkspaceId = }')

    response = requests.get(f"https://api.powerbi.com/v1.0/myorg/groups/{capacityWorkspaceId}/datasets", headers=header)

    datasetId = [dataset.get('id') for dataset in response.json().get('value') if dataset.get('name') == CapacityMetricsDataset][0]
    print(f'{datasetId = }')

    tableList = [{"table": "Capacities"}
                ,{"table": "Workspaces"}
                ,{"table": "TimePoints"}
                ,{"table": "TimePointBackgroundDetail"}
                ,{"table": "Items"}
                ]
    # body = {"objects": tableList} # Need to ask Pat what tables are import and what are direct query
    # body = {"type": "Full", "objects": tableList} # Need to ask Pat what tables are import and what are direct query
    body = {"type": "Full"} # Need to ask Pat what tables are import and what are direct query
    response = requests.post(f"https://api.powerbi.com/v1.0/myorg/groups/{capacityWorkspaceId}/datasets/{datasetId}/refreshes", headers=header, data=json.dumps(body))
    # response = requests.post(f"https://api.powerbi.com/v1.0/myorg/groups/{capacityWorkspaceId}/datasets/{datasetId}/refreshes", headers=header)

    refreshId = response.headers.get('RequestId')
    print(f'{refreshId = } | {response.status_code = }')

    if response.status_code == 202:
        for attempt in range(12): 
            # https://learn.microsoft.com/en-us/power-bi/connect-data/asynchronous-refresh#get-refreshes
            response = requests.get(f"https://api.powerbi.com/v1.0/myorg/groups/{capacityWorkspaceId}/datasets/{datasetId}/refreshes?$top=1", headers=header)
            if response.status_code == 200:
                if response.json().get('value')[0].get('status') != 'Unknown':
                    print(f'Refresh Complete')
                    break
                else:
                    print(f'Refreshing tables ...')
                    time.sleep(20)
            else:
                time.sleep(10)
    else:
        print(f'Refreshed failed - {response.text}')

StatementMeta(, bc92e970-394c-4344-bf51-a281977ce0fd, 39, Finished, Available, Finished)

In [38]:

import requests, math

header = {'Authorization': f'Bearer {mssparkutils.credentials.getToken("pbi")}'
            ,"Content-Type": "application/json"
            }

response = requests.get('https://api.fabric.microsoft.com/v1/workspaces', headers=header)

capacityWorkspaceId = [workspace.get('id') for workspace in response.json().get('value') if workspace.get('displayName') == CapacityMetricsWorkspace][0]
print(f'{capacityWorkspaceId = }')

response = requests.get(f"https://api.powerbi.com/v1.0/myorg/groups/{capacityWorkspaceId}/datasets", headers=header)

datasetId = [dataset.get('id') for dataset in response.json().get('value') if dataset.get('name') == CapacityMetricsDataset][0]
print(f'{datasetId = }')

body = {
  "queries": [
    {
      "query": f"""
        DEFINE
            VAR __DS0Core = 
                SUMMARIZE('Workspaces', 'Workspaces'[WorkspaceName], 'Workspaces'[WorkspaceId])
        EVALUATE
            __DS0Core
    """
    }
  ]
}

response = requests.post(f'https://api.powerbi.com/v1.0/myorg/datasets/{datasetId}/executeQueries', headers=header, json=body )

for retry in range(10):
    workspaceFound = False
    for results in response.json().get('results'):
        for table in results.get('tables'):
            for row in table.get('rows'):
                print(row.get('Workspaces[WorkspaceName]'))
                if row.get('Workspaces[WorkspaceName]') == workspaceName:
                    workspaceFound = True
    if workspaceFound:
        print(f'Workspace "{workspaceName}" was found in the capacity metrics app model')
        break
    else:
        print(f'Workspace "{workspaceName}" was not found. Waiting 5 minutes and will refresh the model and try again...')
        model_refresh()
        time.sleep(300)

StatementMeta(, bc92e970-394c-4344-bf51-a281977ce0fd, 40, Finished, Available, Finished)

capacityWorkspaceId = '0315286f-829f-410f-a0aa-1e45af27297c'
datasetId = 'a3443502-d96b-4b3f-99e5-76920052edad'
WS_DW_Query_Cost
WS_Temp
WS_Demo_Inventory
WS_EventStream_Trigger_Workaround
WS_DW_Query_Cost_Test_Take_Three
WS_DW_Query_Cost_Test_Take_Two
WS_DW_Query_Cost_Test
WS_DW_Query_Cost_Test_Take_Four
WS_DW_Query_Cost_Test_Take_Six
WS_DW_Query_Cost_Test_Take_Eight
WS_TESTING_DEPLOYMENT
WS_DW_Query_Cost_Test_Take_Seven
WS_DW_Query_Cost_Test_Take_Nine
WS_DW_Query_Cost_Test_Take_Five
Workspace "WS_TESTING_DEPLOYMENT" was found in the capacity metrics app model


### Run the notebook

In [39]:
import requests, json, time

header = {'Authorization': f'Bearer {mssparkutils.credentials.getToken("pbi")}'
          ,"Content-Type": "application/json"
          }

body = {
    "executionData": {
        "parameters": {
            "FabricDWWorkspaceName": {"value": workspaceName, "type": "string"}
            ,"FabricDWName": {"value": "WH_SampleData", "type": "string"}
            ,"ConcurrencyNum": {"value": "1", "type": "int"}
            ,"CapacityMetricsWorkspace": {"value": CapacityMetricsWorkspace, "type": "string"}
            ,"CapacityMetricsDataset": {"value": CapacityMetricsDataset, "type": "string"}
        }
    }
}

# print(json.dumps(body, indent=4))

response = requests.request(method='post', url=f'https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/items/{notebookId}/jobs/instances?jobType=RunNotebook', headers=header, data=json.dumps(body))

if response.status_code == 202:
    print('Started Execution of Notebook')
    time.sleep(10) # Sleep for 10 seconds to wait for job to get started. TODO Change in the future to a retry
    print(f"{response.headers=}")
    # jobId = response.headers.get('Location')
    while True:
        # print(f"{response.headers.get('Location')=}")
        responseStatus = requests.request(method="get", url=response.headers.get('Location'), headers=header)
        print(f"Status: {responseStatus.json().get('status')} | {responseStatus.json().get('failureReason') if responseStatus.json().get('failureReason') != None else ''}")
        if responseStatus.json().get('status') in ['Succeeded', 'Failed', 'Completed']:
            print(responseStatus.json().get('status'))
            break
        else:
            time.sleep(120) # Wait 2 minutes to check the status of the run
else:
    print('Error trying to execute')

StatementMeta(, bc92e970-394c-4344-bf51-a281977ce0fd, 41, Finished, Available, Finished)

Started Execution of Notebook
response.headers={'Cache-Control': 'no-store, must-revalidate, no-cache', 'Pragma': 'no-cache', 'Content-Length': '0', 'Content-Type': 'application/octet-stream', 'Content-Encoding': 'gzip', 'Location': 'https://api.fabric.microsoft.com/v1/workspaces/65ba0c17-9fc7-4566-a01a-78da40dbe5f4/items/b3931333-0cb7-4406-9e05-b3ea0ca0f681/jobs/instances/ab140c4e-f16a-48d8-824f-4f0b139bd5a4', 'Retry-After': '60', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains', 'X-Frame-Options': 'deny', 'X-Content-Type-Options': 'nosniff', 'RequestId': 'd515e99d-c849-45f8-a777-3dced09da9ab', 'Access-Control-Expose-Headers': 'RequestId,Location,Retry-After', 'request-redirected': 'true', 'home-cluster-uri': 'https://wabi-west-us3-a-primary-redirect.analysis.windows.net/', 'Date': 'Wed, 25 Sep 2024 13:46:04 GMT'}
Status: InProgress | 
Status: Failed | {'requestId': '7e431a4f-a05e-433f-98ce-fb745eb7fe0b', 'errorCode': 'JobInstanceStatusFailed', 'message': 'Retrieving

In [40]:
# import requests

# response = requests.request(method="get", url=f'https://api.fabric.microsoft.com/v1/workspaces/a3b40eca-afde-448e-bc54-af3d318657b0/items/d20ea153-dbf1-4821-8243-e823bf95bdbb/jobs/instances/237f073e-f403-4d12-a2ae-63377fbe0335', headers=header)
# print(response.status_code)
# print(response.json())


StatementMeta(, bc92e970-394c-4344-bf51-a281977ce0fd, 42, Finished, Available, Finished)